In [1]:
import cv2
import glob
import os
import random
from random import randint
import math
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.utils.np_utils import to_categorical
from keras import optimizers, regularizers
from keras import losses
import h5py

/Users/vikramnagashoka/miniconda3/envs/lane_lines/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Set session

In [2]:
config = tf.ConfigProto()
set_session(tf.Session(config=config))

### Import data

In [7]:
X_train = []
x_label = []
for img_class, directory in enumerate(['red', 'yellow', 'green', 'none']):
    for i, file_name in enumerate(glob.glob("{}/*.jpg".format(directory))):
        file = cv2.imread(file_name)

        file = cv2.cvtColor(file, cv2.COLOR_BGR2RGB);
        resized = cv2.resize(file, (32,32))

        X_train.append(resized/255.)
        x_label.append(img_class)
        
X_train = np.array(X_train)
x_label = np.array(x_label)

In [11]:
labels = to_categorical(x_label)

In [8]:
num_classes = 4
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))

model.add(Conv2D(32, (32, 16), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))

model.add(Conv2D(30, (30, 16), padding='same', activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.01)))

model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(3136, activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.1)))

model.add(Dense(128, activation='relu', kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(0.1)))

model.add(Dense(num_classes, activation='softmax'))

In [9]:
loss = losses.categorical_crossentropy
optimizer = optimizers.Adam()

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, labels, batch_size=32, epochs=40, verbose=True, validation_split=0.1, shuffle=True)

score = model.evaluate(X_train, labels, verbose=0)

print(score)

Train on 330 samples, validate on 37 samples
Epoch 1/40
330/330 [==============================] - 93s - loss: 1564.3367 - acc: 0.3242 - val_loss: 1008.1225 - val_acc: 0.0000e+00
Epoch 2/40
330/330 [==============================] - 99s - loss: 726.7110 - acc: 0.4061 - val_loss: 426.0033 - val_acc: 0.0000e+00
Epoch 3/40
330/330 [==============================] - 93s - loss: 290.6808 - acc: 0.6182 - val_loss: 155.1207 - val_acc: 0.0000e+00
Epoch 4/40
330/330 [==============================] - 92s - loss: 99.9753 - acc: 0.7364 - val_loss: 49.2017 - val_acc: 0.0000e+00
Epoch 5/40
330/330 [==============================] - 99s - loss: 30.2350 - acc: 0.8455 - val_loss: 15.1160 - val_acc: 0.1351
Epoch 6/40
330/330 [==============================] - 87s - loss: 9.3268 - acc: 0.8788 - val_loss: 6.4850 - val_acc: 0.0000e+00
Epoch 7/40
330/330 [==============================] - 101s - loss: 4.1406 - acc: 0.8636 - val_loss: 4.8089 - val_acc: 0.0000e+00
Epoch 8/40
330/330 [========================

In [10]:
model.save('tl_classifier_simulator_one.h5')